In [35]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Load and preprocess the dataset
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images / 255.0
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images / 255.0


# Define the CNN architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), name='conv2d_1'),
    tf.keras.layers.MaxPooling2D((2, 2), name='max_pooling2d_1'),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', name='conv2d_2'),
    tf.keras.layers.MaxPooling2D((2, 2), name='max_pooling2d_2'),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', name='conv2d_3'),
    tf.keras.layers.Flatten(name='flatten'),
    tf.keras.layers.Dense(128, activation='relu', name='dense_1'),
    tf.keras.layers.Dense(10, activation='softmax', name='output')
])
import numpy as np

# Define the weight matrices for each convolutional layer
import numpy as np

# Specify the weights for each layer
conv1_weights = np.array([[ 0.02194263, 0.24223748, 0.6781008 ],
                          [ 0.4244324, -2.0073264, 1.1214318 ],
                          [ 1.2498991, -0.39772153, -0.804002 ]])
conv2_weights = np.array([[-0.07837418, 0.05449237, -0.19679523],
                          [-0.02561542, 0.04406982, -0.02470637],
                          [ 0.03877447, 0.06037814, -0.03606384]])
conv3_weights = np.array([[-0.06434292, 0.0027218, 0.03400089],
                          [ 0.04825743, -0.04293629, 0.01090491],
                          [-0.03814485, -0.07395639, 0.07413089]])

# Set the weights for each layer
model.get_layer("conv2d_1").set_weights([np.expand_dims(conv1_weights, axis=2), np.zeros(32)])
model.get_layer("conv2d_2").set_weights([np.expand_dims(conv2_weights, axis=2), np.zeros(32)])
model.get_layer("conv2d_3").set_weights([np.expand_dims(conv3_weights, axis=2), np.zeros(32)])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
# Print the initial filter matrix
for layer in model.layers:
    if 'conv' in layer.name:
        filters = layer.get_weights()[0]
        print(f"Convolutional matrix of layer {layer.name}:" )
        print(filters[:,:,0,0])

# Define a function to display the convolutional filters as matrices of numbers
def display_convolutional_filters(layer_name):
    layer = model.get_layer(name=layer_name)
    weights = layer.get_weights()[0]
    fig, axs = plt.subplots(nrows=4, ncols=8, figsize=(10, 5))
    fig.suptitle(layer_name)
    for i in range(4):
        for j in range(8):
            axs[i][j].imshow(weights[:, :, 0, i * 8 + j], cmap='gray')
            axs[i][j].axis('off')
            axs[i][j].set_xticklabels([])
            axs[i][j].set_yticklabels([])

# Display the convolutional filters before training
for layer in model.layers:
    if 'conv' in layer.name:
        display_convolutional_filters(layer.name)
plt.show()

# Train the model
history = model.fit(train_images, train_labels, epochs=3, validation_data=(test_images, test_labels))

# Display the convolutional filters after training
for layer in model.layers:
    if 'conv' in layer.name:
        display_convolutional_filters(layer.name)
plt.show()
# Print the initial filter matrix
for layer in model.layers:
    if 'conv' in layer.name:
        filters = layer.get_weights()[0]
        print(f"Convolutional matrix of layer {layer.name}:" )
        print(filters[:,:,0,0])

ValueError: Layer conv2d_1 weight shape (3, 3, 1, 32) is not compatible with provided weight shape (3, 3, 1).

In [36]:
class MyConv2D(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, **kwargs):
        super(MyConv2D, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
    
    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel',
                                      shape=(self.kernel_size, self.kernel_size, input_shape[-1], self.filters),
                                      initializer='glorot_uniform',
                                      trainable=True)
        super(MyConv2D, self).build(input_shape)
    
    def call(self, inputs):
        outputs = tf.nn.conv2d(inputs, self.kernel, strides=[1, 1, 1, 1], padding='SAME')
        outputs = tf.nn.relu(outputs)
        return outputs
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1], input_shape[2], self.filters)


model = tf.keras.models.Sequential([
    MyConv2D(32, 3, activation='relu', input_shape=(28, 28, 1), name='my_conv2d_1'),
    tf.keras.layers.MaxPooling2D((2, 2), name='max_pooling2d_1'),
    MyConv2D(64, 3, activation='relu', name='my_conv2d_2'),
    tf.keras.layers.MaxPooling2D((2, 2), name='max_pooling2d_2'),
    MyConv2D(128, 3, activation='relu', name='my_conv2d_3'),
    tf.keras.layers.Flatten(name='flatten'),
    tf.keras.layers.Dense(128, activation='relu', name='dense_1'),
    tf.keras.layers.Dense(10, activation='softmax', name='output')
])

TypeError: ('Keyword argument not understood:', 'activation')